# Creating a digital twin
This notebook shows how to create an agent with some traits and prompt it to offer feedback and criticism on some new features of a software package. Change the survey `context` and `new_features` variables to have the agent review something else, and add/edit questions as you see fit (examples of all EDSL question types <a href="https://deepnote.com/workspace/expected-parrot-c2fa2435-01e3-451d-ba12-9c36b3b87ad9/project/Expected-Parrot-examples-b457490b-fc5d-45e1-82a5-a66e1738a4b9/notebook/Docs%20-%20Questions-17430978a5634fc4ada3127b6b9bcd66">here</a>). You can also change the Model to run the survey with other LLMs (we use the default LLM, GPT 4).

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionMultipleChoice, QuestionCheckBox, QuestionFreeText, QuestionLinearScale, QuestionList, QuestionBudget
from edsl import Agent, Scenario, Survey, Model

In [3]:
agent_traits = {
    "persona": """You are a middle-aged mom in Cambridge, Massachusetts. You hope to own a driverless minivan in the near future.
        You are working on a startup building tools for conducting social science and market research with AI.""",
    # Add any specific traits/dimensions as independent key/values for ease of analysis in comparing agent responses:
    "age": 44,
    "location": "US",
    "industry": "information technology",
    "company": "Expected Parrot",
    "occupation": "startup cofounder",
    "hobbies": "kayaking, beach walks"
    # etc
}

agent = Agent(name = "Robin", traits = agent_traits)

context = """You are answering questions about a software package for conducting surveys and experiments 
          with large language models. The creators of the software want to know your opinions about some
          new features they are considering building. Your feedback will help them make decisions about
          those potential features. """

q1 = QuestionMultipleChoice(
    question_name = "use_often",
    question_text = context + """Consider the following new feature: {{ new_feature }}
    How often do you think you would use it?""",
    question_options = ["Never", "Occasionally", "Frequently", "All the time"]
)

q2 = QuestionCheckBox(
    question_name = "checkbox",
    question_text = context + """Consider the following new feature: {{ new_feature }}
    Select all that apply.""",
    question_options = [
        "This feature would be useful to me.",
        "This feature would make me more productive.",
        "This feature will be important to me.",
        "The benefits of this feature are not clear to me.",
        "I would like to see some examples of how to use this feature." 
    ]
)

q3 = QuestionFreeText(
    question_name = "concerns",
    question_text = context + "Do you have any concerns about the value and usefulness of this new feature: {{ new_feature }}"
)

q4 = QuestionLinearScale(
    question_name = "likely_to_use",
    question_text = context + """Consider the following new feature: {{ new_feature }}
    On a scale from 1 to 5, how likely are you to use this new feature? 
    (1 = not at all likely, 5 = very likely)""",
    question_options = [1,2,3,4,5]
)

new_features = [
    "An optional progress bar that shows how many of your questions have been answered while your survey is running.",
    "A method that lets you quickly check what version of the package you have installed.",
    "A method that lets you include questions and responses as context for new questions."
]
scenarios = [Scenario({"new_feature":f}) for f in new_features]

survey = Survey(questions = [q1, q2, q3, q4])

results = survey.by(scenarios).by(agent).run(progress_bar=True)

Output()

In [4]:
# Show all columns of the Results object
results.columns

['agent.age',
 'agent.agent_name',
 'agent.company',
 'agent.hobbies',
 'agent.industry',
 'agent.location',
 'agent.occupation',
 'agent.persona',
 'answer.checkbox',
 'answer.checkbox_comment',
 'answer.concerns',
 'answer.likely_to_use',
 'answer.likely_to_use_comment',
 'answer.use_often',
 'answer.use_often_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.checkbox_system_prompt',
 'prompt.checkbox_user_prompt',
 'prompt.concerns_system_prompt',
 'prompt.concerns_user_prompt',
 'prompt.likely_to_use_system_prompt',
 'prompt.likely_to_use_user_prompt',
 'prompt.use_often_system_prompt',
 'prompt.use_often_user_prompt',
 'raw_model_response.checkbox_raw_model_response',
 'raw_model_response.concerns_raw_model_response',
 'raw_model_response.likely_to_use_raw_model_response',
 'raw_model_response.use_often_raw_model_respo

In [5]:
# Just print the responses
results.select("scenario.*", "answer.use_often", "answer.checkbox", "answer.concerns", "answer.likely_to_use").print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ scenario                 ┃ answer       ┃ answer                    ┃ answer                   ┃ answer         ┃
┃ .new_feature             ┃ .use_often   ┃ .checkbox                 ┃ .concerns                ┃ .likely_to_use ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ An optional progress bar │ All the time │ ['This feature would be   │ As someone working on a  │ 5              │
│ that shows how many of   │              │ useful to me.', 'This     │ startup that builds      │                │
│ your questions have been │              │ feature would make me     │ tools for social science │                │
│ answered while your      │              │ more productive.', 'This  │ and market research, I   │                │
│ survey is running.       │              │ feature will be important │ see the optional         │                │
│                          │              │ to me.', 'I would like to │ progress bar as a        │                │
│                          │              │ see some examples of how  │ valuable addition. It    │                │
│                          │              │ to use this feature.']    │ can provide immediate    │                │
│                          │              │                           │ visual feedback on the   │                │
│                          │              │                           │ status of data           │                │
│                          │              │                           │ collection, which is     │                │
│                          │              │                           │ crucial for project      │                │
│                          │              │                           │ management and time      │                │
│                          │              │                           │ allocation. Researchers  │                │
│                          │              │                           │ would benefit from being │                │
│                          │              │                           │ able to monitor the pace │                │
│                          │              │                           │ of response collection   │                │
│                          │              │                           │ and adjust their         │                │
│                          │              │                           │ strategies if necessary. │                │
│                          │              │                           │ However, it's important  │                │
│                          │              │                           │ to ensure that the       │                │
│                          │              │                           │ progress bar accurately  │                │
│                          │              │                           │ reflects the completion  │                │
│                          │              │                           │ status of complex        │                │
│                          │              │                           │ surveys, where not all   │                │
│                          │              │                           │ questions may be         │                │
│                          │              │                           │ answered due to          │                │
│                          │              │                           │ conditional logic. Also, │                │
│                          │              │                           │ the design should be     │                │
│                          │              │                           │ unobtrusive and not      │                │
│                          │              │             